# Import Packages

In [1]:
import pandas as pd
import numpy as np
import turicreate as tc

# Read the Files and Preparing the Data

In [2]:
post = pd.read_csv('posts (1).csv')
user_ratings = pd.read_csv('userRatings.csv', usecols=['user_id', 'post_id', 'Ratings'])

Display the first 5 rows of the post dataset

In [3]:
post.head(5)

,user_id,title,content,tags,slug,created_at,updated_at,image,status_id,action,post_id
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,NaN,what-i-have-learnt-so-far-on-html-985,2019-08-25 19:39:01,2019-08-28 11:30:00,1,NaN,NaN,NaN
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Technology,html-begins-here-4d6,2019-08-25 19:44:36,2019-08-28 11:30:00,1,NaN,NaN,NaN
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,NaN,my-laziness-in-the-open-029,2019-08-25 19:50:33,2019-08-28 11:30:00,1,NaN,NaN,NaN
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,NaN,my-task-2-649,2019-08-25 19:51:36,2019-08-28 11:30:00,1,NaN,NaN,NaN
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str...",NaN,task-2-7e5,2019-08-25 19:51:39,2019-08-28 11:30:00,1,NaN,NaN,NaN


Drop this columns because they are not relevant to our model

In [4]:
col = ['tags', 'slug', 'created_at', 'updated_at', 'image', 'status_id', 'action', 'post_id']

postdf = post.drop(post[col], axis=1)

Read the first 5 cols of the new dataset

In [5]:
postdf.head()

,user_id,title,content
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ..."
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str..."


Display the first 5 rows of the user_ratings dataset

In [6]:
user_ratings.head()

,user_id,post_id,Ratings
0,1,995.0,5
1,1,355.0,2
2,1,996.0,4
3,1,996.0,5
4,1,996.0,4


Merge our postdf and user ratings df

In [7]:
merged_col = pd.merge(postdf, user_ratings, on='user_id')

Display the head of our merged dataframe

In [8]:
merged_col.head()

,user_id,title,content,post_id,Ratings
0,2191,What you have learned so far,"1. I have learnt how to publish and article, ...",143.0,3
1,2191,What you wish to still learn,1. I want more insight on clodinary and so on,143.0,3
2,2191,What you wish should be improved,1. For now am OK with the setting,143.0,3
3,4,Good Afternoon People,Hey,999.0,4
4,4,Good Afternoon People,Hey,278.0,3


In [9]:
#number of users

users = post['user_id'].unique() ; print("number of users" , len(users))

number of users 308


In [10]:
#number of content titles

titles = post['title'].unique() ; print("Number of titles" , len(titles))

Number of titles 658


In [11]:
#train_data, test_data = train_test_split(merged_col, test_size = 0.20, random_state=0)

Convert merged data to csv

In [12]:
merged_col.to_csv('merged_data.csv')

Reading back the csv file and converting it to Sframe for use by the turicreate model

In [13]:
data = tc.SFrame.read_csv('merged_data.csv')

Finished parsing file /home/godwin/Downloads/luci/merged_data.csv

Parsing completed. Parsed 100 lines in 0.04916 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/godwin/Downloads/luci/merged_data.csv

Parsing completed. Parsed 622 lines in 0.04352 secs.

# Model 

Creating a train and validation data for our model

In [14]:
trainind_data, validation_data = tc.recommender.util.random_split_by_user(data, 'user_id', 'title')

Building the model using the ranking factorization recommender

In [15]:
rating_model = tc.ranking_factorization_recommender.create(trainind_data, 
                                                    user_id ='user_id',
                                                   item_id ='title',
                                                   target = 'Ratings')

Preparing data set.

Data has 505 observations with 8 users and 45 items.

Data prepared in: 0.01234s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.52589

Final training RMSE: 0.768353

Creating an item_similarity model, So we can compare the two models together

In [16]:
item_sim_model = tc.item_similarity_recommender.create(trainind_data, user_id='user_id', 
                                                       item_id='content', 
                                                       target='Ratings', similarity_type='cosine')

Warning: Ignoring columns X1, title, post_id;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 505 observations with 8 users and 51 items.

Data prepared in: 0.012382s

Training model from provided data.

Gathering per-item and per-user statistics.

+--------------------------------+------------+

# Compare Both Models

In [17]:
tc.recommender.util.compare_models(validation_data, [rating_model,item_sim_model], metric='rmse')

| Elapsed Time (Item Statistics) | % Complete |

+--------------------------------+------------+

| 439us                          | 100        |

+--------------------------------+------------+

Setting up lookup tables.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.002082s

PROGRESS: Evaluate model M0

Overall RMSE: 0.744626337291712

Per User RMSE (best)
+---------+--------------------+-------+
| user_id |        rmse        | count |
+---------+--------------------+-------+
|    3    | 0.6745149609103928 |   43  |
+---------+--------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+---------+--------------------+-------+
| user_id |        rmse        | count |
+---------+--------------------+-------+
|    1    | 1.1664886704984123 |   2   |
+---------+--------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (best)
+----------------------+---------------------+-------+
|        title         |         rmse        | count |
+----------------------+---------------------+-------+
| How did your day go? | 0.10367495961593762 |   3   |
+----------------------+---------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+--------+--------------------+-------+
| title  |        rmse        | count |
+--------+--------

[{'rmse_by_user': Columns:
  	user_id	int
  	rmse	float
  	count	int
  
  Rows: 4
  
  Data:
  +---------+--------------------+-------+
  | user_id |        rmse        | count |
  +---------+--------------------+-------+
  |    2    | 0.7603050557006685 |   65  |
  |    3    | 0.6745149609103928 |   43  |
  |    1    | 1.1664886704984123 |   2   |
  |    7    | 0.8462973920078615 |   7   |
  +---------+--------------------+-------+
  [4 rows x 3 columns], 'rmse_by_item': Columns:
  	title	str
  	rmse	float
  	count	int
  
  Rows: 29
  
  Data:
  +-------------------------------+---------------------+-------+
  |             title             |         rmse        | count |
  +-------------------------------+---------------------+-------+
  |         Lucid is back         | 0.18978167082379035 |   3   |
  |         Stressful Day         |  0.5465713665134307 |   2   |
  |      Hello New Followers      | 0.15986284255559172 |   2   |
  |      How did your day go?     | 0.103674959615937

We compared both models and we can see that the rating model performed better with a rmse of 0.71

# Recommender System Testing

*Making predictions based on different factors

1. Making recommendation for all users
    * This makes recommendations for all users seen during model creation
    * It returns the first 10 prediction

In [18]:
all_users_rank = rating_model.recommend()
all_users_rank

user_id,title,score,rank
2191,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.796898237992042,1
2191,First,3.580389056917244,2
2191,Took a long while,3.5700966041572033,3
2191,ARE YOU CAPABLE OF BEINGRUTHLESS TO GET AHEAD? ...,3.538916997242683,4
2191,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.493763498593086,5
2191,second,3.475172925371641,6
2191,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.4651693325526653,7
2191,third,3.4600752551145493,8
2191,HOW TO AVOID BEING A ONE-HIT WONDER ...,3.438379227746242,9
2191,DON'T ASK WHY YOU WEREREJECTED ...,3.427877895165199,10


* k=3 arguement means that it should return 3 predictions for each users
* It can basically be any amount of recommendations we want the recommender system to produce

In [19]:
all_users_rank = rating_model.recommend(k=3)
all_users_rank

user_id,title,score,rank
2191,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.796898237992042,1
2191,First,3.580389056917244,2
2191,Took a long while,3.5700966041572033,3
4,ARE YOU CAPABLE OF BEINGRUTHLESS TO GET AHEAD? ...,3.8406760912425457,1
4,SHALLOW THINKING,3.6944144140965878,2
4,TWO YEARS IN THE TRENCHESBUILDING HOTELS.NG ...,3.687869302559608,3
1,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,4.483466840911621,1
1,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.6688995581157147,2
1,EVALUATING STARTUP IDEAS,3.49400156000998,3
7,Took a long while,4.0137413810975495,1


2.Making recommendations for new users without any information
    * created a new user with user_id = 20000
    * asked the rec_sys to recommend 3 articles for the user

In [20]:
rank_factor = rating_model.recommend(users=[20000], k=3)
rank_factor

user_id,title,score,rank
20000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.8835690092332302,1
20000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.875557489324325,2
20000,ARE YOU CAPABLE OF BEINGRUTHLESS TO GET AHEAD? ...,3.7747896086461483,3


* it can also return a list of users

In [21]:
rank_factor = rating_model.recommend(users=[20000, 21000, 22000], k=3)
rank_factor

user_id,title,score,rank
20000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.8835690092332302,1
20000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.875557489324325,2
20000,ARE YOU CAPABLE OF BEINGRUTHLESS TO GET AHEAD? ...,3.7747896086461483,3
21000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.8835690092332302,1
21000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.875557489324325,2
21000,ARE YOU CAPABLE OF BEINGRUTHLESS TO GET AHEAD? ...,3.7747896086461483,3
22000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.8835690092332302,1
22000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.875557489324325,2
22000,ARE YOU CAPABLE OF BEINGRUTHLESS TO GET AHEAD? ...,3.7747896086461483,3


In [28]:
rating_model.save('rating_model.model')

# Follower Recommender System

Read the data

In [22]:
following = pd.read_csv('nooffollowers (2).csv', usecols=['user_id', 'name', 'username', 'email', 'no of followers'])
following.head()

,user_id,name,username,email,no of followers
0,1,Eniayomi Oluwaseyi,eniayomi,oluwaseyieniayomi@gmail.com,2
1,2,Elijah Okokon,DMatrix,okoelijah@gmail.com,1
2,3,Jeffrey Ogah,jeff.ogah,jeff.ogah@gmail.com,8
3,4,Oluwaseyi Oluwapelumi,nathan,nathanoluwaseyi@gmail.com,10
4,6,Seyi Onifade,xyluz,hngtechapps@gmail.com,1


Convert the data into Sframe

In [23]:
following = tc.SFrame(following)
following.head()

user_id,name,username,email,no of followers
1,Eniayomi Oluwaseyi,eniayomi,oluwaseyieniayomi@gmail.com ...,2
2,Elijah Okokon,DMatrix,okoelijah@gmail.com,1
3,Jeffrey Ogah,jeff.ogah,jeff.ogah@gmail.com,8
4,Oluwaseyi Oluwapelumi,nathan,nathanoluwaseyi@gmail.com,10
6,Seyi Onifade,xyluz,hngtechapps@gmail.com,1
7,Mark Essien,markessien,markessien@gmail.com,3
8,Alex Moses,ahlesswywk,ahlesswywk@gmail.com,3
9,Edmund Ekott,26th_edmund,edmund.timfon@gmail.com,8
11,Jenni Essien,j.naiaretti,j.naiaretti@gmail.com,1
15,Patrick Aziken,onaziken,onaziken@gmail.com,2


# Build the followers Recommender System
* We used the factorization model so our recommender system does not become bias to recommending the most popular followers to all users.

In [24]:
model_item_sim = tc.factorization_recommender.create(following, 
                                          user_id='user_id', 
                                          item_id='name', 
                                          target='no of followers')

Preparing data set.

Data has 905 observations with 905 users and 905 items.

Data prepared in: 0.013737s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.327737

Final training RMSE: 0.572457

# Testing the Followers Recommender System

1. Testing for all users using by recommending 3 users to our users.

In [25]:
rec_result_factorization = model_item_sim.recommend(k=3)
rec_result_factorization

user_id,name,score,rank
1,Timothy Mayor,6.648975433302185,1
1,Chiamaka Honor Osuji,6.645798803758881,2
1,M,6.540994466734192,3
2,ukadike ochem,5.950322227072975,1
2,Amadi Onengiye Williams,5.94320401687648,2
2,Chiz-Ugeh Macximillian.I. ...,5.608111755442879,3
3,Amadi Onengiye Williams,7.599663139772675,1
3,Omolola Richard,7.150131048155091,2
3,David Adedeji,7.12925851483371,3
4,Amadi Onengiye Williams,7.746184946489594,1


 2. Making recommendations for new users

In [26]:
rec_result_factorization_newUsers = model_item_sim.recommend(users=[20000], k=3)
rec_result_factorization_newUsers

user_id,name,score,rank
20000,Chiz-Ugeh Macximillian.I. ...,6.602134765577576,1
20000,ukadike ochem,6.443364800405762,2
20000,Amadi Onengiye Williams,6.350936712217591,3


In [29]:
model_item_sim.save('follower_rec.model')